In [1]:
%pwd

'd:\\Project\\ETE_Projects\\image_classifer\\Mushroom_Edibility_Classifier\\research'

In [2]:
import os 

os.chdir("../")

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbackConfig:
    root_dir: Path
    checkpoint_dir: Path 
    tensorboard_dir: Path 
    
    

In [4]:
from Mushroom_edibility_classifier.constants import *
from Mushroom_edibility_classifier.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def PrepareCallbackConfig(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_path)
        tensorboard_dir = Path(config.tensorboard_dir)
        root_dir = Path(self.config.artifacts_root)

        create_directories([model_checkpoint_dir, tensorboard_dir])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(root_dir),
            checkpoint_dir=Path(model_checkpoint_dir),
            tensorboard_dir=Path(tensorboard_dir)
        )

        return prepare_callback_config

In [26]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time



In [ ]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(self.config.checkpoint_dir,"model.keras"),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [56]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.PrepareCallbackConfig()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2025-06-05 21:18:35,298: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-05 21:18:35,310: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-05 21:18:35,316: INFO: common: created directory at: artifacts]
[2025-06-05 21:18:35,316: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-06-05 21:18:35,326: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_logs]


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=artifacts\prepare_callbacks\checkpoint_dir